In [11]:
import pandas as pd

In [12]:
num_interactions_dict = {
    100_000: '100k',
    500_000: '500k',
    1_000_000: '1M',
    5_000_000: '5M',
    15_000_000: '15M',
    30_000_000: '30M'
}

num_items_dict = {
    1_000: '1k',
    10_000: '10k',
    100_000: '100k'
}

In [13]:
algorithms_list = [
    'mab2rec_LinUCB',
    'mab2rec_LinTS',
    'mab2rec_LinGreedy',
    'gobrec_GPU_LinUCB',
    'gobrec_GPU_LinTS',
    'gobrec_GPU_LinGreedy',
    'gobrec_CPU_LinUCB',
    'gobrec_CPU_LinTS',
    'gobrec_CPU_LinGreedy',
    'iRec_LinUCB',
    'iRec_LinTS',
    'iRec_LinGreedy'
]

In [14]:

from utils.constants import RESULTS_SAVE_PATH
import os
import numpy as np

def get_experiment_mean_and_std(exp_type: str, algo_name: str, num_interactions_num: int, num_interactions_str: str, num_items_num: int, num_items_str: str):
    df = pd.read_csv(os.path.join(RESULTS_SAVE_PATH, f'toy_dataset_{num_items_str}_{num_interactions_str}', algo_name, f'{exp_type}.csv'))
    mean = df['total'].mean()
    std = df['total'].std()
    return mean, std

multipliers_dict = {
    'mab2rec': 1,
    'gobrec_GPU': 1 / 200,
    'gobrec_CPU': 1 / 150,
    'iRec': 0.95,

    'LinUCB': 1.2,
    'LinGreedy': 1,
    'LinTS': 1.3,

    'not_incremental': 1,
    'incremental': 1.2
}

def get_experiment_mean_and_std(exp_type: str, algo_name: str, num_interactions_num: int, num_interactions_str: str, num_items_num: int, num_items_str: str):
    mean = num_interactions_num * num_items_num / 1_000_000
    
    for mult_name, mult in multipliers_dict.items():
        if mult_name in algo_name or mult_name in exp_type:
            mean *= mult
    
    std = mean * np.random.uniform(0.01, 0.05)  # Assuming a random variation for std
    return mean, std

In [ ]:
import plotly.express as px

for num_items_num, num_items_str in num_items_dict.items():
    data = []
    for num_interactions_num, num_interactions_str in num_interactions_dict.items():
        for algo_name in algorithms_list:
            mean, std = get_experiment_mean_and_std('not_incremental', algo_name, num_interactions_num, num_interactions_str, num_items_num, num_items_str)
            data.append({
                'Algorithm': algo_name,
                'Num_interactions': num_interactions_num,
                'Mean Execution Time (s)': mean,
                'Standard Deviation (s)': std
            })
        
    df = pd.DataFrame(data)
    fig = px.line(df, x='Num_interactions', y='Mean Execution Time (s)', color='Algorithm',
                    error_y='Standard Deviation (s)', title=f'Execution Time for {num_items_str} items and {num_interactions_str} interactions')
    fig.show()

## 1. Time comparision tables

In [ ]:

def generate_time_comparision_table_header():
    return '''
\\begin{table}[htpb]
    \centering
    \\begin{tabular}{l|cccc|cccc|cccc}

    & \multicolumn{4}{c|}{LinGreedy} & \multicolumn{4}{c|}{LinUCB} & \multicolumn{4}{c}{LinTS} \\\\

    \hline
    
    & \makecell{Time\\\\(m)} & \makecell{Opt.\\\\mab2rec} & \makecell{Opt.\\\\iRec} & \makecell{Opt.\\\\CPU} & \makecell{Time\\\\(m)} & \makecell{Opt.\\\\mab2rec} & \makecell{Opt.\\\\iRec} & \makecell{Opt.\\\\CPU} & \makecell{Time\\\\(m)} & \makecell{Opt.\\\\mab2rec} & \makecell{Opt.\\\\iRec} & \makecell{Opt.\\\\CPU} \\\\

    \hline
    '''

def generate_time_comparision_table_dataset_separation(num_items_num: int, num_items_str: str, num_interactions_num: int, num_interactions_str: str):
    return ' & \multicolumn{12}{c}{' + f'Toy dataset with {num_items_num:,} ({num_items_str}) items and {num_interactions_num:,} ({num_interactions_str}) interactions' + '} \\\\\n\n\t\hline\n'

def generate_time_comparision_table_footer(label: str, caption_prefix: str = '', caption_posfix: str = ''):
    return f'''
    \end{{tabular}}
    \caption{{{caption_prefix}``Opt. mab2rec'' columns indicates how many times the specified algorithm was faster than the mab2rec implementation. ``Opt. iRec'' columns indicates how many times the specified algorithm was faster than the iRec implementation. ``Opt. CPU'' indicates how many times the specified algorithm was faster than the gobrec CPU implementation.{caption_posfix}}}
    \label{{{label}}}
\end{{table}}
    '''

test_str = generate_time_comparision_table_header() + generate_time_comparision_table_dataset_separation(1000, '1k', 5_000_000, '5M') + generate_time_comparision_table_footer('test_label')

with open('test_table.tex', 'w') as f:
    f.write(test_str)